In [1]:
import os

os.chdir("../..")

In [2]:
import utils
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
import seaborn as sns
import pickle
from sklearn.metrics import roc_auc_score

In [4]:
results = pd.read_csv("output/results_ptbxl.csv", index_col="Method")
results = results['exp4_AUC'].to_frame()
results = results[results['exp4_AUC']!='---']
results['AUC'] = results['exp4_AUC'].str.extract(r'(\d+\.\d+)', expand=False).astype(float)
results['CI'] = results['exp4_AUC'].str.extract(r'\((\d+)\)', expand=False).astype(float)
results['CI'] = results['CI']/1000

results.sort_index(inplace=True)
results.index = results.index.str.replace(r'models\\', '')
results = results.sort_values("AUC")

C:\Users\agarw\AppData\Local\Temp\ipykernel_38324\2379245209.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  results.index = results.index.str.replace(r'models\\', '')


In [5]:
methods = results.index.to_list()
methods.remove("naive")

In [6]:
raw_test_preds = []
raw_val_preds = []

for method in methods:
    test = np.load(f"output/exp4/models/{method}/y_test_pred.npy", allow_pickle=True)
    val = np.load(f"output/exp4/models/{method}/y_val_pred.npy", allow_pickle=True)
    raw_test_preds += [test]
    raw_val_preds += [val]

In [7]:
with open(r"C:\Mohit\Imperial\fyp-hearts\output\exp4\data\mlb.pkl", 'rb') as f:
    mlb = pickle.load(f).classes_
y_test =np.load(r"C:\Mohit\Imperial\fyp-hearts\output\exp4\data\y_test.npy", allow_pickle=True)
y_val =np.load(r"C:\Mohit\Imperial\fyp-hearts\output\exp4\data\y_val.npy", allow_pickle=True)

In [36]:
list(mlb).index("SR")

37

In [10]:
y_test = pd.DataFrame(y_test, columns=mlb)
attnet_test = pd.DataFrame((raw_test_preds[-1]>0.5).astype(int), columns=mlb)

In [16]:
from sklearn.metrics import f1_score, roc_auc_score,accuracy_score, recall_score, precision_score

print(f"f1 score: {f1_score(y_test['AFIB'],(raw_test_preds[-1][:,3]>0.5).astype(int))}")
print(f"AUC: {roc_auc_score(y_test['AFIB'],(raw_test_preds[-1][:,3]>0.5).astype(int))}")
print(f"Accuracy: {accuracy_score(y_test['AFIB'],(raw_test_preds[-1][:,3]>0.5).astype(int))}")
print(f"Specificity: {recall_score(y_test['AFIB'],(raw_test_preds[-1][:,3]>0.5).astype(int), pos_label=0)}")
print(f"Sensitivity: {recall_score(y_test['AFIB'],(raw_test_preds[-1][:,3]>0.5).astype(int), pos_label=1)}")
print(f"Precision: {precision_score(y_test['AFIB'],(raw_test_preds[-1][:,3]>0.5).astype(int))}")

f1 score: 0.8111888111888113
AUC: 0.877069929332349
Accuracy: 0.9748603351955307
Specificity: 0.9909819639278558
Sensitivity: 0.7631578947368421
Precision: 0.8656716417910447


In [42]:
raw_test_preds[method][:,[3,37]]

array([[0.04559132, 0.8319306 ],
       [0.09970009, 0.7726198 ],
       [0.05880202, 0.8108702 ],
       ...,
       [0.05014538, 0.83230096],
       [0.05335931, 0.8276611 ],
       [0.07478154, 0.8011249 ]], dtype=float32)

In [57]:
test_aucs = pd.DataFrame(columns=methods, index=mlb)


true = y_test['AFIB']
for method in range(len(methods)):
    pred = raw_test_preds[method][:,3]
    test_aucs[methods[method]][mlb[3]] = roc_auc_score(true, pred)

test_aucs = test_aucs.dropna().T
test_aucs['F1 Score'] = 0
test_aucs['Accuracy'] = 0
test_aucs['Sensitivity'] = 0
test_aucs['Specificity'] = 0
test_aucs['Precision'] = 0



for method in range(len(methods)):
    pred = raw_test_preds[method][:,3]
    test_aucs["F1 Score"][methods[method]] = f1_score(true, (pred>0.5).astype(int), average='macro')
    test_aucs["Accuracy"][methods[method]] = accuracy_score(true, (pred>0.5).astype(int))
    test_aucs["Sensitivity"][methods[method]] = recall_score(true, (pred>0.5).astype(int), pos_label=1, average='macro')
    test_aucs["Specificity"][methods[method]] = recall_score(true, (pred>0.5).astype(int), pos_label=0, average='macro')
    test_aucs["Precision"][methods[method]] = precision_score(true, (pred>0.5).astype(int), average='macro')
    

test_aucs

C:\Users\agarw\AppData\Local\Temp\ipykernel_38324\202060508.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_aucs["F1 Score"][methods[method]] = f1_score(true, (pred>0.5).astype(int), average='macro')
C:\Users\agarw\AppData\Local\Temp\ipykernel_38324\202060508.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_aucs["Accuracy"][methods[method]] = accuracy_score(true, (pred>0.5).astype(int))
C:\Users\agarw\AppData\Local\Temp\ipykernel_38324\202060508.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydat

,AFIB,F1 Score,Accuracy,Sensitivity,Specificity,Precision
Simple+Baseline+lead1,0.711209,0.481660,0.929236,0.500000,0.500000,0.464618
Wavelet+NN+lead1,0.82162,0.517845,0.928771,0.517983,0.517983,0.696577
Wavelet+RF+lead1,0.803156,0.488145,0.929236,0.503039,0.503039,0.714818
InceptionSE+lead1,0.950289,0.724502,0.945996,0.670084,0.670084,0.854750
NewWavelet+NN+lead1,0.920482,0.716073,0.945531,0.660716,0.660716,0.860131
VGGNet+lead1,0.959346,0.836723,0.958101,0.828545,0.828545,0.845413
LeNetAttn+lead1,0.927477,0.710157,0.936220,0.676979,0.676979,0.764697
Wavelet+NN,0.932572,0.723212,0.943669,0.674909,0.674909,0.824660
LeNet+lead1,0.912279,0.648619,0.934358,0.609120,0.609120,0.768606
Simple+Baseline,0.833091,0.615353,0.935289,0.579231,0.579231,0.807757


In [55]:
y = np.array([raw_test_preds[i][:,3] for i in range(len(raw_test_preds))]).mean(axis=0)

# print([f1_score(y_test[['AFIB']], (y>i/100).astype(int), average='macro') for i in range(50,51)])
f1_score(y_test[['AFIB']], (y>0.5).astype(int))

0.47524752475247517

In [59]:
test_aucs = pd.DataFrame(columns=methods, index=mlb)


true = y_test[['AFIB', 'SR']]
for method in range(len(methods)):
    pred = raw_test_preds[method][:,[3,37]]
    test_aucs[methods[method]][mlb[37]] = roc_auc_score(true, pred)

test_aucs = test_aucs.dropna().T
test_aucs['F1 Score'] = 0
test_aucs['Accuracy'] = 0
test_aucs['Sensitivity'] = 0
test_aucs['Specificity'] = 0
test_aucs['Precision'] = 0



for method in range(len(methods)):
    pred = raw_test_preds[method][:,[3,37]]
    test_aucs["F1 Score"][methods[method]] = f1_score(true, (pred>0.5).astype(int), average='macro')
    test_aucs["Accuracy"][methods[method]] = accuracy_score(true, (pred>0.5).astype(int))
    test_aucs["Sensitivity"][methods[method]] = recall_score(true, (pred>0.5).astype(int), pos_label=1, average='macro')
    test_aucs["Specificity"][methods[method]] = recall_score(true, (pred>0.5).astype(int), pos_label=0, average='macro')
    test_aucs["Precision"][methods[method]] = precision_score(true, (pred>0.5).astype(int), average='macro')
    

test_aucs

C:\Users\agarw\AppData\Local\Temp\ipykernel_38324\4197251568.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_aucs["F1 Score"][methods[method]] = f1_score(true, (pred>0.5).astype(int), average='macro')
C:\Users\agarw\AppData\Local\Temp\ipykernel_38324\4197251568.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_aucs["Accuracy"][methods[method]] = accuracy_score(true, (pred>0.5).astype(int))
C:\Users\agarw\AppData\Local\Temp\ipykernel_38324\4197251568.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.py

,SR,F1 Score,Accuracy,Sensitivity,Specificity,Precision
Simple+Baseline+lead1,0.663581,0.437843,0.778864,0.499701,0.499701,0.389613
Wavelet+NN+lead1,0.788456,0.482232,0.797020,0.508088,0.508088,0.640955
Wavelet+RF+lead1,0.781745,0.449958,0.794227,0.498809,0.498809,0.651306
InceptionSE+lead1,0.842936,0.668516,0.739758,0.600268,0.600268,0.812270
NewWavelet+NN+lead1,0.872431,0.681652,0.817970,0.638488,0.638488,0.815163
VGGNet+lead1,0.901571,0.806045,0.837989,0.818505,0.818505,0.795958
LeNetAttn+lead1,0.837016,0.668786,0.784451,0.667189,0.667189,0.697155
Wavelet+NN,0.86703,0.688408,0.811453,0.654637,0.654637,0.777214
LeNet+lead1,0.846205,0.611185,0.795158,0.594224,0.594224,0.712585
Simple+Baseline,0.733594,0.571510,0.777933,0.569095,0.569095,0.737936


In [71]:
methods2 = ['VGGNet+lead1', 'LeNet+lead1', 'AlexNet+lead1', 'ResNet+Baseline+lead1', 'AttNet+Baseline+lead1']
idxs = np.where(np.isin(methods, methods2))[0]

In [72]:
m = np.array([raw_test_preds[i][:,[3,37]] for i in idxs]).mean(axis=0)

f1_score(true, (m>0.5).astype(int), average='macro')
# m.shape

0.854894856493322